In [144]:
import pandas as pd
import numpy as np
import re
# Read the CSV file
df = pd.read_csv('datasets/cleaned_14.csv')

# View the first 5 rows
df.head()

,Product Desc,Qty,Total Price,Total Cost,Total Cost %,Total Profit,Total Profit %,all_nan,Branch,Department,Category,Division
0,1 SHOT DECAFE,404.00,0.00,"3,856.85",0.00,"-3,856.85",100.00,False,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
1,2 SHOT DECAFE,637.00,0.00,"12,162.45",0.00,"-12,162.45",100.00,False,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
2,3 SHOT DECAFE,113.00,0.00,"3,236.32",0.00,"-3,236.32",100.00,False,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
3,ADD BANANA SAUCE LARGE,4.00,266.67,139.97,52.49,126.69,47.51,False,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
4,ADD BANANA SAUCE MEDIUM,47.00,"1,487.39",822.34,55.29,665.04,44.71,False,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION


In [145]:
#just to check the number of branches if it matches the given
len(df["Branch"].unique())

25

In [146]:
df = df.drop(columns=["all_nan"])
df

,Product Desc,Qty,Total Price,Total Cost,Total Cost %,Total Profit,Total Profit %,Branch,Department,Category,Division
0,1 SHOT DECAFE,404.00,0.00,"3,856.85",0.00,"-3,856.85",100.00,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
1,2 SHOT DECAFE,637.00,0.00,"12,162.45",0.00,"-12,162.45",100.00,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
2,3 SHOT DECAFE,113.00,0.00,"3,236.32",0.00,"-3,236.32",100.00,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
3,ADD BANANA SAUCE LARGE,4.00,266.67,139.97,52.49,126.69,47.51,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
4,ADD BANANA SAUCE MEDIUM,47.00,"1,487.39",822.34,55.29,665.04,44.71,Bir Hasan,TAKE AWAY,BEVERAGES,COLD BAR SECTION
...,...,...,...,...,...,...,...,...,...,...,...
13138,ADD TUNA,7.00,"1,027.03",334.50,32.57,692.53,67.43,Stories kaslik,TAKE AWAY,FOOD,SUBS
13139,Total By Division:,51.00,"5,579.58","2,381.77",42.69,"3,197.81",57.31,Stories kaslik,TAKE AWAY,FOOD,SUBS
13140,Total By Category:,"10,528.50","1,287,075.68","484,453.55",37.64,"802,622.12",62.36,Stories kaslik,TAKE AWAY,FOOD,NaN
13141,Total By Department:,"22,252.50","2,818,715.92","844,897.47",29.97,"1,973,818.45",70.03,Stories kaslik,TAKE AWAY,NaN,NaN


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13143 entries, 0 to 13142
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Product Desc    13143 non-null  object
 1   Qty             13143 non-null  object
 2   Total Price     13143 non-null  object
 3   Total Cost      13143 non-null  object
 4   Total Cost %    13143 non-null  object
 5   Total Profit    13143 non-null  object
 6   Total Profit %  13143 non-null  object
 7   Branch          13143 non-null  object
 8   Department      13118 non-null  object
 9   Category        13076 non-null  object
 10  Division        12996 non-null  object
dtypes: object(11)
memory usage: 1.1+ MB


In [148]:
# there are objects where they should be floats so we will clean 
def to_num(x):
    if pd.isna(x): 
        return np.nan
    if isinstance(x, (int, float, np.number)):
        return float(x)
    s = str(x).strip().replace(",", "")
    if re.match(r"^\(.*\)$", s):  # (123.45) -> -123.45
        s = "-" + s[1:-1]
    try:
        return float(s)
    except:
        return np.nan
    
for c in ["Qty", "Total Price", "Total Cost", "Total Cost %", "Total Profit", "Total Profit %"]:
    df[c] = df[c].apply(to_num)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13143 entries, 0 to 13142
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Product Desc    13143 non-null  object 
 1   Qty             13143 non-null  float64
 2   Total Price     13143 non-null  float64
 3   Total Cost      13143 non-null  float64
 4   Total Cost %    13143 non-null  float64
 5   Total Profit    13143 non-null  float64
 6   Total Profit %  13143 non-null  float64
 7   Branch          13143 non-null  object 
 8   Department      13118 non-null  object 
 9   Category        13076 non-null  object 
 10  Division        12996 non-null  object 
dtypes: float64(6), object(5)
memory usage: 1.1+ MB


### We notice there is some null values in category let's justify and fill them properly

In [150]:
#let's fix the total price column since it is given that there is some bug

df["Total Price"] = (
    pd.to_numeric(df["Total Cost"], errors="coerce").fillna(0)
    + pd.to_numeric(df["Total Profit"], errors="coerce").fillna(0)
)

In [151]:
(df["Total Price"] == 0).sum() # these could be gifts ...etc that we re going to assume they don't affect anything

np.int64(1318)

In [152]:
df = df[df['Total Price'] != 0]

In [153]:
# 1. Find the index of the first row where the value is NA
idx = df[df['Division'].isna()].index[0]

# 2. Slice from that index to index + 6 (to get the NA row + 5 ahead)
result = df.iloc[idx -3 : idx + 6]

result

,Product Desc,Qty,Total Price,Total Cost,Total Cost %,Total Profit,Total Profit %,Branch,Department,Category,Division
260,BLUEBERRY MUFFIN,168.0,34421.68,7423.84,21.57,26997.84,78.43,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
261,BROWNIES CAKE,751.0,119998.50,49290.63,41.08,70707.87,58.92,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
262,CARROT CAKE,167.0,48688.11,16540.79,33.97,32147.32,66.03,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
263,CHOCOLATE SABLE,222.0,52959.82,26492.00,50.02,26467.82,49.98,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
264,DOUBLE CHOCOLATE MUFFIN,267.0,54666.49,12538.07,22.94,42128.42,77.06,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
265,FRAMBOISE CHEESE CAKE,130.0,47193.69,31530.85,66.81,15662.84,33.19,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
266,LAZY CAKE,533.0,170541.92,79505.83,46.62,91036.09,53.38,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
267,LOTUS CHEESE CAKE,155.0,56488.29,37374.25,66.16,19114.04,33.84,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY
268,ORANGE CAKE,84.0,25470.00,5118.51,20.10,20351.49,79.90,Bir Hasan,TAKE AWAY,FOOD,COFFEE PASTRY


### we notice that whenever there is a change in the division or category or department some of the others are left NaN since already Total By division is not a food ...etc
### Next let's make another dataset that discusses the revenues of each division and each category across everything

In [154]:
import pandas as pd

# df = pd.read_csv("cleaned_14.csv")  # or your file
desc_col = "Product Desc"

is_total = df[desc_col].astype(str).str.strip().str.match(r"(?i)^total\s+by\s+")
df_totals = df[is_total].copy()
df_items  = df[~is_total].copy()

df_items.reset_index(drop=True, inplace=True)
df_totals.reset_index(drop=True, inplace=True)


In [155]:
## let's check if it is clean:
col0 = df.columns[0]  # first column

mask = df_items[col0].astype(str).str.contains("Total by", case=False, na=False)
len(df_items.loc[mask])

0

In [156]:
df_totals[col0].unique()

array(['Total By Division:', 'Total By Category:', 'Total By Department:',
       'Total By Branch:'], dtype=object)

In [159]:
#getting the division:
target = "Total By Division:"
division_total = df_totals.loc[df_totals[col0].astype(str).str.contains(target, case=False, na=False)].drop(col0, axis=1)
target = "Total By Category:"
category_total = df_totals.loc[df_totals[col0].astype(str).str.contains(target, case=False, na=False)].drop(col0, axis=1)
target = "Total By Department:"
department_total = df_totals.loc[df_totals[col0].astype(str).str.contains(target, case=False, na=False)].drop(col0, axis=1)
target = "Total By Branch:"
branch_total = df_totals.loc[df_totals[col0].astype(str).str.contains(target, case=False, na=False)].drop(col0, axis=1)

## Margin Analysis 
we will define margin in this case as Sales - COGs

In [160]:
# here sales will be just toatl price and cogs will be the total cost and GM = total profit
#so we will aggregate and do our actual margin analysis

g = (category_total.groupby(["Branch","Category"], dropna=False)
       .agg(Sales=("Total Price","sum"),
            COGS=("Total Cost","sum"),
            GM=("Total Profit","sum"),
            Qty=("Qty","sum"))
       .reset_index())

g["GM_pct"] = g["GM"] / g["Sales"]

In [163]:
import plotly.express as px

# optional: ignore tiny sales so the heatmap reflects meaningful business
g_plot = g[g["Sales"] > 100000].copy()

heat = g_plot.pivot(index="Branch", columns="Category", values="GM_pct")

fig = px.imshow(
    heat,
    aspect="auto",
    text_auto=".2%",
    labels=dict(x="Category", y="Branch", color="GM%")
)

fig.update_layout(
    title="Gross Margin % Heatmap (Branch × Category)",
    height=700
)

fig.show()

## This tells us that have roughly that margins are consistent across branches for both categories 

In [166]:
## let's do the same for department:

g = (department_total.groupby(["Branch","Department"], dropna=False)
       .agg(Sales=("Total Price","sum"),
            COGS=("Total Cost","sum"),
            GM=("Total Profit","sum"),
            Qty=("Qty","sum"))
       .reset_index())

g["GM_pct"] = g["GM"] / g["Sales"]


g_plot = g[g["Sales"] > 100000].copy()

heat = g_plot.pivot(index="Branch", columns="Department", values="GM_pct")

fig = px.imshow(
    heat,
    aspect="auto",
    text_auto=".2%",
    labels=dict(x="Department", y="Branch", color="GM%")
)

fig.update_layout(
    title="Gross Margin % Heatmap (Branch × Department)",
    height=700
)

fig.show()

## We notice that aley is not performing take away service although others are and they are performing good so this pushes us to study the effect of integrating takeaway in alay

### we notice that toters are performing well which pushes us to say that we should implement toters on other branches
### but the real question is if we add Toters to more branches, will incremental sales and GM$ increase?